In [1]:
print('importing')
import pandas as pd, numpy as np, torch


import sys
import argparse
from dvae.learning_algo import LearningAlgorithm
from dvae.learning_algo_ss import LearningAlgorithm_ss
import os
import shutil
import socket
import datetime
import pickle
import numpy as np
import torch
import matplotlib.pyplot as plt
from dvae.utils import myconf, get_logger, loss_ISD, loss_KLD, loss_MPJPE
from dvae.dataset import h36m_dataset, speech_dataset
from dvae.model import build_VAE, build_DKF, build_STORN, build_VRNN, build_SRNN, build_RVAE, build_DSAE

importing


In [2]:

# STORN, SRNN, vanilla AE

dep_train = pd.read_csv('depressed_train.csv')
dep_val = pd.read_csv('depressed_val.csv')
healthy = pd.read_csv('healthy.csv')


In [3]:

model = torch.load('/home/harrison_kintsugihello_com/speech_final_project/saved_model/depressed/WSJ0_2021-12-09-08h31_SRNN_z_dim=16/SRNN_checkpoint.pt')





class Options:
	def __init__(self):
		self.parser = argparse.ArgumentParser()
		self.opt = None

	def _initial(self):
		# Basic config file
		self.parser.add_argument('--cfg', type=str, default='/home/harrison_kintsugihello_com/speech_final_project/config/speech/cfg_srnn_depressed.ini', help='config path')
		# Schedule sampling
		self.parser.add_argument('--ss', action='store_true', help='schedule sampling')
		self.parser.add_argument('--use_pretrain', action='store_true', help='if use pretrain')
		self.parser.add_argument('--pretrain_dict', type=str, default=None, help='pretrained model dict')
		# Resume training
		self.parser.add_argument('--reload', action='store_true', help='resume the training')
		self.parser.add_argument('--model_dir', type=str, default=None, help='model directory')

	def get_params(self):
		self._initial()
		self.opt = self.parser.parse_args()
		params = vars(self.opt)
		return params


params = {
	'cfg': '/home/harrison_kintsugihello_com/speech_final_project/config/speech/cfg_srnn_depressed.ini',
	'ss': False,
	'use_pretrain': False,
	'reload': False,
	'model_dir': None,
}


learning_algo = LearningAlgorithm(params=params)

learning_algo.build_model()


In [4]:

learning_algo.model.load_state_dict(model['model_state_dict'])

<All keys matched successfully>

In [5]:

model = learning_algo.model


In [6]:
model

SRNN(
  (activation): Tanh()
  (mlp_x_h): Sequential(
    (linear0): Linear(in_features=513, out_features=256, bias=True)
    (activation0): Tanh()
    (dropout0): Dropout(p=0.0, inplace=False)
  )
  (rnn_h): LSTM(256, 128)
  (mlp_hx_g): Sequential(
    (linear0): Linear(in_features=641, out_features=256, bias=True)
    (activation0): Tanh()
    (dropout0): Dropout(p=0.0, inplace=False)
  )
  (rnn_g): LSTM(256, 128)
  (mlp_gz_z): Sequential(
    (linear0): Linear(in_features=144, out_features=64, bias=True)
    (activation0): Tanh()
    (dropout0): Dropout(p=0.0, inplace=False)
    (linear1): Linear(in_features=64, out_features=32, bias=True)
    (activation1): Tanh()
    (dropout1): Dropout(p=0.0, inplace=False)
  )
  (inf_mean): Linear(in_features=32, out_features=16, bias=True)
  (inf_logvar): Linear(in_features=32, out_features=16, bias=True)
  (mlp_hz_z): Sequential(
    (linear0): Linear(in_features=144, out_features=64, bias=True)
    (activation0): Tanh()
    (dropout0): Dropou

In [7]:
from IPython.display import Audio
import torchaudio

train_ex, val_ex = dep_train.filename[0], dep_val.filename[0]
healthy_ex_1, healthy_ex_2 = healthy.filename[0], healthy.filename[1]

train_ex, sr = torchaudio.load(f'/home/harrison_kintsugihello_com/data/native_16_trim/{train_ex}.wav')
val_ex, sr = torchaudio.load(f'/home/harrison_kintsugihello_com/data/native_16_trim/{val_ex}.wav')
healthy_ex_1, sr = torchaudio.load(f'/home/harrison_kintsugihello_com/data/native_16_trim/{healthy_ex_1}.wav')
healthy_ex_2, sr = torchaudio.load(f'/home/harrison_kintsugihello_com/data/native_16_trim/{healthy_ex_2}.wav')
train_ex


tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0012, 0.0007, 0.0005]])

In [8]:

Audio(healthy_ex_2.squeeze(), sr)




OSError: [Errno 9] Bad file descriptor

In [47]:
healthy_ex_2.squeeze().numpy()

array([-0.0007354 , -0.00244492, -0.00347459, ..., -0.01331043,
        0.02859789, -0.03174061], dtype=float32)